In [671]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [672]:
df = pd.read_csv('street.csv')

In [673]:
df.head()

,Street,Ward,District,Prices,Area,toFace,type,certificate,width,length,floors,Location
0,Hà Mục,Hòa Thọ Đông,cẩm lệ,-0.109935,0.331876,Tây Nam,Nhà mặt tiền,Không có,-0.384833,0.532496,4,"16.01918,108.19537"
1,Thi Sách,Hòa Thuận Tây,hải châu,-0.504485,0.010225,Không,Nhà mặt tiền,Không có,1.214177,-0.987467,1,"16.0546,108.20547"
2,Ông Ích Khiêm,Tam Thuận,thanh khê,0.974593,-0.375756,Tây,Nhà mặt tiền,Giấy tờ hợp lệ,-0.839577,0.198567,3,"16.06653,108.2142"
3,Núi Thành,Hòa Cường Bắc,hải châu,1.469697,1.125282,Tây,Nhà mặt tiền,Giấy tờ hợp lệ,0.486759,0.358670,3,"16.03194,108.22279"
4,Trương Văn Hiến,Khuê Mỹ,ngũ hành sơn,0.371615,0.117442,Đông Nam,Nhà mặt tiền,Giấy tờ hợp lệ,-0.384833,0.250511,4,"16.01947,108.25592"


In [674]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14612 entries, 0 to 14611
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Street       14612 non-null  object 
 1   Ward         14612 non-null  object 
 2   District     14612 non-null  object 
 3   Prices       14612 non-null  float64
 4   Area         14612 non-null  float64
 5   toFace       14612 non-null  object 
 6   type         14612 non-null  object 
 7   certificate  14612 non-null  object 
 8   width        14612 non-null  float64
 9   length       14612 non-null  float64
 10  floors       14612 non-null  int64  
 11  Location     14610 non-null  object 
dtypes: float64(4), int64(1), object(7)
memory usage: 1.3+ MB


## Lable Encoding

In [675]:
df['District'].unique()

array(['cẩm lệ', 'hải châu', 'thanh khê', 'ngũ hành sơn', 'sơn trà',
       'liên chiểu', 'hòa vang'], dtype=object)

Chuyển dữ liệu trường District về dạnh số

- cẩm lệ -> 0
- hải châu -> 1
- thanh khê -> 2
- ngũ hành sơn -> 3
- sơn trà -> 4
- liên chiểu -> 5
- hòa vang -> 6

In [676]:
from sklearn.preprocessing import LabelEncoder

# Khởi tạo LabelEncoder
le = LabelEncoder()

df['District'] = le.fit_transform(df['District'])


In [677]:
type_map = {'Nhà trong hẻm':0,'Nhà mặt tiền':1}
df['type']=le.fit_transform(df['type'].map(type_map))

In [678]:
df['certificate']= le.fit_transform(df['certificate'])

In [679]:
df.head(5)

,Street,Ward,District,Prices,Area,toFace,type,certificate,width,length,floors,Location
0,Hà Mục,Hòa Thọ Đông,0,-0.109935,0.331876,Tây Nam,1,1,-0.384833,0.532496,4,"16.01918,108.19537"
1,Thi Sách,Hòa Thuận Tây,2,-0.504485,0.010225,Không,1,1,1.214177,-0.987467,1,"16.0546,108.20547"
2,Ông Ích Khiêm,Tam Thuận,6,0.974593,-0.375756,Tây,1,0,-0.839577,0.198567,3,"16.06653,108.2142"
3,Núi Thành,Hòa Cường Bắc,2,1.469697,1.125282,Tây,1,0,0.486759,0.358670,3,"16.03194,108.22279"
4,Trương Văn Hiến,Khuê Mỹ,4,0.371615,0.117442,Đông Nam,1,0,-0.384833,0.250511,4,"16.01947,108.25592"


In [680]:
df[df.Location.isnull()]

# Xóa các cột có giá trị NaN trong cột "Location"
df = df.drop(df[df.Location.isnull()].index)

In [681]:
tttp = (16.061203,108.226744)
beach = (16.0704,108.2461)
airport = (16.043889 ,108.199444)
def distance(l1,l2):
    # return (np.sqrt((l1[0]-l2[0])**2+(l1[1]-l2[1])**2)*1000)
    lat1_rad = np.radians(l1[0])
    lon1_rad = np.radians(l1[1])
    lat2_rad = np.radians(l2[0])
    lon2_rad = np.radians(l2[1])

    # Tính khoảng cách
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad
    a = np.sin(delta_lat / 2) ** 2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(delta_lon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    radius = 6371  # Bán kính trái đất (đơn vị: kilômét)
    distance = radius * c
    return distance

df['Latitude'] = (df['Location'].str.split(',',expand=True)[0])
df['Longitude'] = df['Location'].str.split(',',expand=True)[1]

In [682]:
df['Latitude']=df['Latitude'].astype('float32')
df['Longitude']=df['Longitude'].astype('float32')

In [683]:
df['DistanceToCenter'] = distance(tttp,(np.float64(df['Latitude']),np.float64(df['Longitude'])))
df['DistanceToBeach']= distance(beach,(np.float64(df['Latitude']),np.float64(df['Longitude'])))
df['DistanceToAirport']= distance(airport,(np.float64(df['Latitude']),np.float64(df['Longitude'])))

# Chia Train/Test

In [684]:
# streets = ['Cao Thắng','Lê Đình Lý','Hùng Vương','Lê Lợi','3.2','Nguyễn Chí Thanh','Phan Châu Trinh','Tiểu La','2.9','Quang Trung','Đống Đa','Nguyễn Du','Lý Tự Trọng','Hoàng Diệu','Ông Ích Khiêm','Nguyễn Hoàng','Thái Phiên','Lê Hồng Phong','Hoàng Văn Thụ','Yên Bái','Trưng Nữ Vương','Ngô Gia Tự','Triệu Nữ Vương','Cao Thắng','Bắc Đẩu','Hải Hồ','Pasteur','Lê Đình Dương','Nguyễn Văn Linh','Lê Thanh Nghị','Xô Viết Nghệ Tĩnh','Trần Phú','Bạch Đằng','Nguyễn Tất Thành','Thái Thị Bôi','Lê Độ','Trường Chinh','Hải Phòng','Điện Biên Phủ','Hà Huy Tập','Trần Cao Vân','Nguyễn Tri Phương','Lê Lợi','Hùng Vương','Lê Đình Lý','Duy Tân','Tiểu La','Nguyễn Phước Nguyên','Xô Viết Nghệ Tĩnh','2.9','Nguyễn Lương Bằng','Nguyễn tất Thành','Bàu Tràm','Âu Cơ','Đoàn Phú Thứ','Nam Cao','Lạc Long Quân','Nguyễn Sinh Sắc','Kinh Dương Vương','Nguyên Chánh','Trần Đình Tri','Hoàng Văn Thái','Tố Hữu','Tôn Đức Thắng','Yết Kiêu','Lý Tử Tấn','Hoàng Sa','Lê Đức Thọ','Phan Bá Phiến','Trần Nhân Tông','Trương Định','Chu Huy Mân','Nại Hiên Đông','Dương Lâm','Lê Văn Duyệt','Dương Vân Nga','Lê Chân','Lý Đạo Hành','Ngô Quyền','Lê Văn Thứ','Võ Nguyên Giáp','Hồ Nghinh','Nguyễn Công Trứ','Võ Văn Kiệt','Lê Hữu Trác','Nguyễn Văn Thoại','Trần Hưng Đạo','Lê Văn Duyệt','Võ Nguyên Giáp','Lê Quang Đạo','Châu Thị Vĩnh tế','Phan Tứ','Trần Văn Dư','Hồ Xuân Hương','Bà Huyện Thanh Quan','Võ Như Hưng','Hoài Thanh','An Dương Vương','Phạm Hữu Kính','Lê Văn Hưu','Hồ Huân Nghiệp','Nguyễn Lữ','Đoàn Khuê','Trịnh Lỗi','Nghiêm Xuân yêm','Minh Mạng','Nguyễn Xiến','Trường Sa','Nguyễn Duy Trinh','Mai Đăng Chơn','Võ Chí Công','Phạm Đức Nam','Lê Quát','Trường Chinh','Tôn Đản','Lê Trọng Tấn','Lê Đại Hành','Xô Viết Nghệ Tĩnh','Thăng Long','Phạm Tứ','Phan Khôi','Lê Quang Định','Trần Nam Trung','Võ Chí Công','Hồ Sĩ Dương','Quốc lộ 14B']
# streets = [str(x).lower() for x in streets]
# df['Street']=df['Street'].str.lower()
# mask = df['Street'].str.contains('|'.join(streets))
# df['isBigRoad'] = mask
# df['isBigRoad']=le.fit_transform(df['isBigRoad'])

In [685]:
from sklearn.model_selection import train_test_split
house = df[['Prices','District','Area','type','certificate','width','length','floors']]
df_train, df_test = train_test_split(house, test_size = 0.3, random_state=42)

print(df_train.shape)
print(df_test.shape)

(10227, 8)
(4383, 8)


In [686]:
X_train = df_train.copy()
X_test = df_test.copy()

print("X_train: {0}".format(X_train.shape))
print("X_test: {0}".format(X_test.shape))

X_train: (10227, 8)
X_test: (4383, 8)


In [687]:
y_train = X_train.pop('Prices')
y_test = X_test.pop('Prices')
print("Y_train: {0}".format(y_train.shape))
print("y_test: {0}".format(y_test.shape))

Y_train: (10227,)
y_test: (4383,)


In [688]:
# streets = ['Cao Thắng','Lê Đình Lý','Hùng Vương','Lê Lợi','3.2','Nguyễn Chí Thanh','Phan Châu Trinh','Tiểu La','2.9','Quang Trung','Đống Đa','Nguyễn Du','Lý Tự Trọng','Hoàng Diệu','Ông Ích Khiêm','Nguyễn Hoàng','Thái Phiên','Lê Hồng Phong','Hoàng Văn Thụ','Yên Bái','Trưng Nữ Vương','Ngô Gia Tự','Triệu Nữ Vương','Cao Thắng','Bắc Đẩu','Hải Hồ','Pasteur','Lê Đình Dương','Nguyễn Văn Linh','Lê Thanh Nghị','Xô Viết Nghệ Tĩnh','Trần Phú','Bạch Đằng','Nguyễn Tất Thành','Thái Thị Bôi','Lê Độ','Trường Chinh','Hải Phòng','Điện Biên Phủ','Hà Huy Tập','Trần Cao Vân','Nguyễn Tri Phương','Lê Lợi','Hùng Vương','Lê Đình Lý','Duy Tân','Tiểu La','Nguyễn Phước Nguyên','Xô Viết Nghệ Tĩnh','2.9','Nguyễn Lương Bằng','Nguyễn tất Thành','Bàu Tràm','Âu Cơ','Đoàn Phú Thứ','Nam Cao','Lạc Long Quân','Nguyễn Sinh Sắc','Kinh Dương Vương','Nguyên Chánh','Trần Đình Tri','Hoàng Văn Thái','Tố Hữu','Tôn Đức Thắng','Yết Kiêu','Lý Tử Tấn','Hoàng Sa','Lê Đức Thọ','Phan Bá Phiến','Trần Nhân Tông','Trương Định','Chu Huy Mân','Nại Hiên Đông','Dương Lâm','Lê Văn Duyệt','Dương Vân Nga','Lê Chân','Lý Đạo Hành','Ngô Quyền','Lê Văn Thứ','Võ Nguyên Giáp','Hồ Nghinh','Nguyễn Công Trứ','Võ Văn Kiệt','Lê Hữu Trác','Nguyễn Văn Thoại','Trần Hưng Đạo','Lê Văn Duyệt','Võ Nguyên Giáp','Lê Quang Đạo','Châu Thị Vĩnh tế','Phan Tứ','Trần Văn Dư','Hồ Xuân Hương','Bà Huyện Thanh Quan','Võ Như Hưng','Hoài Thanh','An Dương Vương','Phạm Hữu Kính','Lê Văn Hưu','Hồ Huân Nghiệp','Nguyễn Lữ','Đoàn Khuê','Trịnh Lỗi','Nghiêm Xuân yêm','Minh Mạng','Nguyễn Xiến','Trường Sa','Nguyễn Duy Trinh','Mai Đăng Chơn','Võ Chí Công','Phạm Đức Nam','Lê Quát','Trường Chinh','Tôn Đản','Lê Trọng Tấn','Lê Đại Hành','Xô Viết Nghệ Tĩnh','Thăng Long','Phạm Tứ','Phan Khôi','Lê Quang Định','Trần Nam Trung','Võ Chí Công','Hồ Sĩ Dương','Quốc lộ 14B']
# streets = [str(x).lower() for x in streets]
# df['Street']=df['Street'].str.lower()
# mask = df['Street'].str.contains('|'.join(streets))
# df['isBigRoad'] = mask
# df['isBigRoad']=le.fit_transform(df['isBigRoad'])

In [689]:
from sklearn.model_selection import train_test_split
house = df[['Prices','District','Area','type','certificate','width','length','floors','DistanceToCenter','DistanceToBeach']]
df_train, df_test = train_test_split(house, test_size = 0.2, random_state=42)

print(df_train.shape)
print(df_test.shape)

(11688, 10)
(2922, 10)


In [690]:
X_train = df_train.copy()
X_test = df_test.copy()

print("X_train: {0}".format(X_train.shape))
print("X_test: {0}".format(X_test.shape))

X_train: (11688, 10)
X_test: (2922, 10)


In [691]:
y_train = X_train.pop('Prices')
y_test = X_test.pop('Prices')
print("Y_train: {0}".format(y_train.shape))
print("y_test: {0}".format(y_test.shape))

Y_train: (11688,)
y_test: (2922,)


# Random Forest Regressor (QKhang)

In [692]:
from sklearn.ensemble import RandomForestRegressor
model_RD = RandomForestRegressor(random_state=13)
model_RD.fit(X_train, y_train)

RandomForestRegressor(random_state=13)

In [696]:
from sklearn.model_selection import cross_validate
model_RD = RandomForestRegressor(random_state=13)
scores = cross_validate(model_RD,X_train,y_train)
scores.mean()

{'fit_time': array([3.96714997, 3.885005  , 3.80036855, 3.79137778, 3.78875709]),
 'score_time': array([0.05088067, 0.05132246, 0.05375075, 0.06159186, 0.05221009]),
 'test_score': array([0.81562628, 0.81698237, 0.81219745, 0.80990551, 0.81028331])}

In [693]:
predictions_RD = model_RD.predict(X_test)


In [694]:
from sklearn.metrics import r2_score 
r2_score_RD = r2_score(y_test, predictions_RD)
print(r2_score_RD)

0.8384770151761002


In [ ]:
from sklearn.model_selection import cross_validate

In [695]:
rs = {
  "Price_Test": y_test[:10],
  "Price_Predict": predictions_RD[:10]
}

df_temp = pd.DataFrame(rs)
print(df_temp)

       Price_Test  Price_Predict
3976     1.383244       1.079963
11582    1.469697       1.463862
1454    -0.176286      -0.205672
9724    -0.062432       0.116033
10383    1.774793       1.356611
5039    -0.571794      -0.627608
1618     1.774793       1.493907
7282    -0.594973      -0.191256
2484    -0.822411      -0.885853
3502     0.027266       0.026926
